In [1]:
# Import Libraries
import time
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

In [2]:
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Cam\.wdm\drivers\chromedriver\win32\95.0.4638.54]


In [3]:
#URL Mars News Site
url = 'https://redplanetscience.com/'
browser.visit(url)

time.sleep(1)

In [4]:
# --- create HTML object ---
html = browser.html

In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')
#soup

In [6]:
#collect the latest News Title and Paragraph Text
latest_news_title = soup.find("div", class_ = "content_title").text
print(latest_news_title)
news_pgraph=soup.find("div", class_ = "article_teaser_body").text
print(news_pgraph)

6 Things to Know About NASA's Ingenuity Mars Helicopter
The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.




### JPL Mars Space Images - Featured Image


In [7]:
#URL JPL Mars Space Image
jpl_mars_url = 'https://spaceimages-mars.com/'
browser.visit(jpl_mars_url)

time.sleep(1)
# --- create HTML object ---
html = browser.html

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
image_soup = bs(html, 'html.parser')
#image_soup

In [9]:
# find the image url to the full size .jpg and save a complete url string for this image
featured_image = image_soup.find("img", class_ = "headerimage fade-in")
featured_image_url = jpl_mars_url + featured_image["src"]
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg



### Mars Facts

In [25]:
#URL Mars Facts
marsfacts_url = 'https://space-facts.com/mars/'
browser.visit(marsfacts_url)

time.sleep(1)
# --- create HTML object ---
html = browser.html
#html

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57346): Max retries exceeded with url: /session/dda93c936a28c99932412be3b1101d92/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018296FD9A90>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_table = pd.read_html(html)
#mars_table

In [ ]:
mfacts_df = mars_table[0]
mfacts_df.columns =['Description','Value']
mfacts_df

In [ ]:
#Use Pandas to convert the data to a HTML table string.
mfacts_df.to_html("mars_facts.html",index =False)
mfacts = mfacts_df.to_html(header=True,index=False)
print(mfacts)

### Mars Hemispheres

In [24]:
# Visit the astrogeology site
marshems_url = "http://marshemispheres.com/"
browser.visit(marshems_url)

time.sleep(1)
# --- create HTML object ---
html = browser.html
# html

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57346): Max retries exceeded with url: /session/dda93c936a28c99932412be3b1101d92/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018296F2E7C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
marshem_soup = bs(html, 'html.parser')
# marshem_soup

In [15]:
#link is located in "div" class "description"
divs_desc = marshem_soup.find_all("div", class_ = "description")
#divs_desc

In [16]:
hemis = []
for h in divs_desc:
    hems_html = h.find("a")["href"]
    hemis.append(hems_html)
#hemis

In [17]:
# Create list of dictionaries called hemisphere_image_urls
hemisphere_image_urls = []

# Iterate through all URLs saved in variable
for h in hemis:
    
    mars_hem_url = marshems_url + h
    #print(mars_hem_url)
    
    # Visit to url
    browser.visit(mars_hem_url)
    #time.sleep(1)
    # HTML Object
    html = browser.html

     # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')

    # Find all full resolution image for all Hemisphere URLs
    img_url = soup.find("img", class_ = "wide-image")["src"]
    
    # Find all titles 
    title = soup.find("h2", class_ = "title").text
    hemis_title = title.split(" Enhanced")[0] #removed the enhanced word
    #print(hemis_title)
    
    # Append image&title to the dict
    hemisphere_image_urls.append({"title": hemis_title, "img_url": marshems_url + img_url})

# Exit Browser
browser.quit()



In [18]:
hemisphere_image_urls

[]



        MARS Data

In [ ]:
#Create Dictionary for all Mars Data
Mars_Data ={}
#Append all Scrapped Data
Mars_Data['latest news title'] = latest_news_title
Mars_Data['news paragraph'] = news_pgraph
Mars_Data['featured image url'] = featured_image_url
Mars_Data['mars facts'] = mfacts
Mars_Data['mars hemispheres urls'] = hemisphere_image_urls

In [20]:
Mars_Data

{'latest news title': "6 Things to Know About NASA's Ingenuity Mars Helicopter",
 'news paragraph': 'The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.',
 'featured image url': 'https://spaceimages-mars.com/image/featured/mars3.jpg'}

In [21]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [22]:
#Insert object into MongoDB
client.mars_db.mars_data.insert_one(Mars_Data)